# Objective
Is there a relationship between wager amounts and win/loss records?

In [1]:
from db import fetch
from tqdm import tqdm

In [2]:
query = """
select userid, contestid, matchmaking_settings_amounts.value, contests.winner, contests.__updatetime

from matchmaking

inner join

matchmaking_settings_amounts

on matchmaking_settings_amounts.matchmaking_settings_id = matchmaking.id

inner join

contests

on contests.id = matchmaking.contestid

LIMIT 10000
"""

In [3]:
df = fetch(query)
df.shape

(10000, 5)

In [4]:
df.head()

,userid,contestid,value,winner,__updatetime
0,3111566670926342,1003039290741578,1,4.442767e+15,2019-03-08 11:20:41.036
1,4442767398411702,1003039290741578,1,4.442767e+15,2019-03-08 11:20:41.036
2,4152796729003125,1005392136747638,1,NaN,2019-03-09 11:27:00.162
3,4377362426913185,1005509244463165,5,4.377362e+15,2019-03-08 11:20:41.036
4,1145254476848614,1007614208671572,1,1.821939e+15,2019-03-08 11:28:00.352


In [18]:
lookup = {}

for _, row in tqdm(df.iterrows()):
    u = row['userid'] 
    v = row['value']
    t = row['__updatetime']
    w = row['winner']
    s = row['userid'] == row['winner']
    
    
    if u in lookup.keys():
        if s:
            lookup[u].append((t, "W", v))
        else:
            lookup[u].append((t, "L", v))
    else:
        if s:
            lookup[u] = [(t, "W", v)]
        else:
            lookup[u] = [(t, "L", v)]

10000it [00:01, 7214.58it/s]


In [19]:
# sort games by timestamp
for k, v in lookup.items():
    lookup[k] = sorted(v, key=lambda x: x[0])

In [20]:
# remove timestamp
for k, v in lookup.items():
    lookup[k] = list(map(lambda x: (x[1], x[2]), v))

In [24]:
path

{'LL': 0, 'LW': 0, 'WW': 0, 'WL': 0}

In [26]:
lookup

{3111566670926342: [('L', '1')],
 4442767398411702: [('W', '1'), ('W', '1')],
 4152796729003125: [('L', '1'), ('W', '1'), ('W', '1')],
 4377362426913185: [('W', '5')],
 1145254476848614: [('W', '1'), ('L', '1')],
 1821938569681467: [('W', '1'), ('W', '1')],
 2595037316349710: [('L', '5')],
 8718029712625980: [('L', '5'),
  ('L', '10'),
  ('L', '10'),
  ('W', '1'),
  ('W', '10')],
 6624699373973908: [('L', '1')],
 2376355977608851: [('W', '1')],
 4557377313101968: [('L', '1'),
  ('L', '1'),
  ('W', '1'),
  ('L', '5'),
  ('L', '5'),
  ('L', '1'),
  ('L', '1'),
  ('L', '1'),
  ('W', '5'),
  ('W', '1'),
  ('W', '1'),
  ('L', '1'),
  ('W', '1')],
 1751595923347444: [('L', '1')],
 6342972652677534: [('L', '5')],
 2582538124383220: [('W', '5')],
 1723370409639342: [('L', '5'), ('L', '5'), ('W', '10'), ('W', '10')],
 4935287555412355: [('L', '5'), ('W', '10'), ('W', '5')],
 6260129694421364: [('L', '1')],
 1692921899784176: [('L', '1')],
 3496099105947534: [('L', '1')],
 2247665294321348: [('L

In [55]:
counter = 0

path = {"LL":0, "LW":0, "WW":0, "WL":0}


for k, v in lookup.items():
    if len(v) > 3:
        # print("userid {}: record {}".format(k, v))
        for i in range(1, len(v)):
            counter += 1
            if v[i][1] > v[i-1][1]:
                foo = v[i-1][0] + v[i][0]
                path[foo] += 1

In [56]:
path

{'LL': 89, 'LW': 78, 'WW': 110, 'WL': 84}

In [51]:
110+84

194

In [52]:
89+78

167

In [57]:
sum(list(path.values()))

361

In [59]:
(194/361)/(167/361)

1.1616766467065867